In [7]:
import pickle

import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from module.pipeline import best_parameters, retrained
from module.features import pipe_1, pipe_2, pipe_3, pipe_4, pipe_5, pipe_6, pipe_7,\
                     pipe_8, pipe_9, pipe_10, pipe_11, pipe_12, pipe_13, pipe_14,\
                     pipe_15

### オリジナルデータの呼び出し

In [8]:
df = pd.read_csv('./data/train.csv')

### 

### パイプライン候補からデータセットの作成

In [9]:
to_pipe ={
            'df': df,
            'split_kwrg': {},
            'train_flg': True,
            'retrain': True,
            }
# 特徴量候補を設定
pipe_lines = [
            pipe_1,  # base 
            # pipe_2,  # StSlopeCat
            # pipe_3,  # CholsetMean
            # pipe_4,  # AgeCat
            # pipe_5,  # StSlopeCat CholetMean AgeCat
            # pipe_6,  # RestingBpCat
            # pipe_7,  # OldPeakCat
            # pipe_8,  # RestingBpCat OldPeakCat
            pipe_9,  # Onehot
            pipe_10,  # CholestMean AgeCat Onehot
            pipe_11,  # CholCut
            pipe_12,  # CholCut Onehot,
            pipe_13,  # DropByShap
            pipe_14,  # CholCatDropBy,
            pipe_15  # CholRegression
            ]
data_set = {pipe.__name__: pipe(**to_pipe) for pipe in pipe_lines}

                                Base(pipe_1)                               


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0
3,34,1,115,0,1,154,0,0.2
4,54,0,160,201,0,163,0,0.0


                               Onehot(pipe_9)                              


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0
3,34,1,115,0,1,154,0,0.2,1,0,0,0,0,1,0,0,0,1
4,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1


                     CholestMean AgeCat Onehot(pipe_10)                    


...

,Age,Sex,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ST_Slope_Flat,ST_Slope_Up
0,56,1,342,1,150,3.0,1,1,0
1,55,0,394,0,150,0.0,0,0,1
2,47,1,0,1,120,0.0,0,1,0
3,34,1,0,1,154,0.2,1,0,1
4,54,0,201,0,163,0.0,0,0,1


                        CholCut DropByShap(pipe_14)                        


,Age,Sex,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ST_Slope_Flat,ST_Slope_Up
0,56,1,342,1,150,3.0,1,1,0
1,55,0,394,0,150,0.0,0,0,1
2,54,0,201,0,163,0.0,0,0,1
3,64,1,335,0,158,0.0,0,0,1
4,65,1,254,0,127,2.8,1,1,0


                          Chol Regression(pipe_15)                         


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,230,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0
3,34,1,115,248,1,154,0,0.2,1,0,0,0,0,1,0,0,0,1
4,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1


### グリットサーチで訓練したモデルの呼び出し

In [10]:
file_name = 'models'
with open(f'./data/{file_name}.pkl', 'rb') as f:
    trained_models = pickle.load(f)

### 最適なパラメータで再訓練

In [23]:
retrain = [
            XGBClassifier,
            DecisionTreeClassifier,
            SVC,
            KNeighborsClassifier
            ]

best_param = best_parameters(trained_models, pipe_lines)
# 再訓練してpickleで保存
retrained_models = retrained(retrain, pipe_lines, data_set, best_param, file_name)